<a href="https://colab.research.google.com/github/PiyawatFrom/practices/blob/main/Week_5_Unsupervised_Learning_Association_rules_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Association rules

Find frequent itemsets and association rules from the dataset.
Answer the following questions
- How many itemsets of length 1 if we set the `min_support=0.005` and `min_confidence=0.5`
- How many items in the longest itemsets?
- How many rules can we extract from the data if we set the `min_support=0.005` and `min_confidence=0.5`

Upload your colab notebook and the screenshot of the rules to Google form.

## Loading the dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Around 2 min.
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx')

In [ ]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


## Data cleansing

- Clear any entries with N/A.
- Convert the `StockCode` column type to `string`.
    - `df['StockCode'].astype('string')`
- Choose only invoices in `United Kingdom`.
    - Hint: There are `379423` invoices.
- Create a list of StockCodes in each invoice. We will pass this to the Apriori algorithm.
    - Hint: Transction `489434` has the following stock code list `['85048', '79323P', '79323W', '22041', '21232', '22064', '21871', '21523']`
- Set the support to `0.005` and confidence to `0.5`
    - You may try with different numbers. However, small support means more samples required to make the pattern be **interesting**.

In [ ]:
# Drop N/A
df2 = df.dropna()
# Convert 'StockCode' column to 'string'
df2['StockCode'] = df2['StockCode'].astype(str)
# Choose only rows where 'Country' is 'United Kingdom'
df_uk = df2.loc[df2['Country'] == 'United Kingdom']
df_uk

<ipython-input-4-34f51d032d14>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['StockCode'] = df2['StockCode'].astype(str)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [ ]:
# Create a list to stock codes for each invoice
transctions = df_uk.groupby('Invoice')['StockCode'].apply(tuple).tolist()
#Hint: Transction `489434` has the following stock code list
#    ['85048', '79323P', '79323W', '22041', '21232', '22064', '21871', '21523']
transctions[0]

('85048', '79323P', '79323W', '22041', '21232', '22064', '21871', '21523')

In [ ]:
!pip install efficient-apriori==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from efficient_apriori import apriori

In [ ]:
# pass the transaction list to apriori algorithm, set the min_support at min_confidence
# e.g. if
# - trans_list is the transaction list variable
# - min_support is 0.05
# - min_confidence is 0.5
# Uncomment the following line

#สำหรับตอบข้อ 1 , 2 min_support=0.005 และ min_confidence=0.5
itemsets, rules = apriori(transctions, min_support = 0.005, min_confidence = 0.5)

In [ ]:
rules

[{15056N} -> {15056BL},
 {15056BL} -> {15056N},
 {20679} -> {15056BL},
 {15056P} -> {15056N},
 {20679} -> {15056N},
 {20674} -> {20675},
 {20674} -> {20676},
 {20674} -> {20677},
 {20675} -> {20676},
 {20677} -> {20675},
 {20675} -> {20677},
 {20675} -> {21240},
 {21244} -> {20675},
 {20677} -> {20676},
 {21242} -> {20676},
 {20677} -> {21239},
 {21243} -> {20677},
 {20711} -> {85099B},
 {20712} -> {85099B},
 {20713} -> {85099B},
 {20717} -> {20718},
 {20719} -> {20724},
 {20719} -> {22355},
 {20723} -> {20724},
 {22356} -> {20724},
 {20726} -> {20725},
 {20727} -> {20725},
 {20728} -> {20725},
 {22384} -> {20725},
 {20967} -> {20969},
 {20970} -> {20969},
 {20969} -> {20970},
 {20971} -> {20972},
 {20973} -> {20974},
 {20975} -> {20974},
 {20974} -> {20975},
 {20983} -> {20982},
 {20982} -> {20983},
 {21078} -> {21080},
 {21086} -> {21080},
 {21094} -> {21080},
 {21094} -> {21086},
 {21086} -> {21094},
 {21122} -> {21121},
 {21121} -> {21122},
 {21124} -> {21121},
 {21121} -> {21124},

In [ ]:
itemsets

{1: {('85048',): 251,
  ('79323P',): 301,
  ('79323W',): 425,
  ('22041',): 127,
  ('21232',): 1350,
  ('22064',): 317,
  ('21871',): 335,
  ('21523',): 474,
  ('22350',): 156,
  ('22349',): 201,
  ('22195',): 289,
  ('22353',): 269,
  ('48173C',): 284,
  ('21755',): 941,
  ('21754',): 1146,
  ('84879',): 1267,
  ('22119',): 234,
  ('22142',): 128,
  ('22296',): 468,
  ('22295',): 412,
  ('22109',): 243,
  ('22107',): 120,
  ('22194',): 109,
  ('82582',): 334,
  ('21181',): 855,
  ('21756',): 285,
  ('21333',): 185,
  ('84596F',): 119,
  ('22111',): 830,
  ('22145',): 155,
  ('22130',): 265,
  ('21364',): 125,
  ('21351',): 108,
  ('35400',): 348,
  ('37370',): 246,
  ('10002',): 214,
  ('20703',): 135,
  ('21987',): 169,
  ('21989',): 229,
  ('84970S',): 702,
  ('20971',): 546,
  ('22271',): 406,
  ('22272',): 221,
  ('22274',): 333,
  ('21912',): 323,
  ('22112',): 775,
  ('21329',): 208,
  ('21100',): 147,
  ('21033',): 188,
  ('20711',): 406,
  ('21411',): 318,
  ('84032A',): 244,


In [ ]:
#Ans 1
print(len(itemsets[1]))

989


In [ ]:
# Ans 2
for i in range(5) :
    print('itemset member =' , i+1 )
    print(itemsets[i+1].values())
    print('max freq in',i+1,'member','=', max(itemsets[i+1].values()))
    print('_'*60)

itemset member = 1
dict_values([251, 301, 425, 127, 1350, 317, 335, 474, 156, 201, 289, 269, 284, 941, 1146, 1267, 234, 128, 468, 412, 243, 120, 109, 334, 855, 285, 185, 119, 830, 155, 265, 125, 108, 348, 246, 214, 135, 169, 229, 702, 546, 406, 221, 333, 323, 775, 208, 147, 188, 406, 318, 244, 280, 921, 688, 428, 3010, 178, 280, 113, 138, 156, 399, 166, 247, 289, 754, 322, 145, 392, 372, 540, 531, 193, 209, 276, 411, 598, 162, 966, 642, 156, 260, 778, 459, 142, 618, 791, 693, 617, 1625, 532, 639, 252, 1218, 786, 694, 492, 396, 830, 1027, 381, 219, 930, 353, 252, 275, 273, 270, 263, 126, 1001, 678, 618, 1315, 126, 170, 213, 148, 212, 489, 440, 329, 290, 116, 390, 212, 279, 693, 117, 111, 501, 504, 277, 276, 802, 537, 189, 339, 247, 409, 531, 467, 284, 178, 259, 601, 131, 300, 143, 184, 142, 377, 216, 159, 169, 141, 273, 128, 174, 202, 935, 531, 209, 148, 218, 176, 264, 199, 356, 442, 275, 673, 397, 408, 712, 275, 202, 427, 488, 510, 219, 169, 116, 163, 155, 142, 152, 214, 113, 137, 135,

In [ ]:
# Codes to find the answer of the following questions
# - How many itemsets of length 1 if we set the min_support=0.005 and min_confidence=0.5
# - How many items in the longest itemsets?
# - How many rules can we extract from the data if we set the min_support=0.005 and min_confidence=0.5

#สำหรับตอบข้อ 3 min_support=0.05 and min_confidence=0.5
itemsets2, rules2 = apriori(transctions, min_support = 0.05, min_confidence = 0.5)

In [ ]:
itemsets2

{1: {('21232',): 1350,
  ('21754',): 1146,
  ('84879',): 1267,
  ('85123A',): 3010,
  ('85099B',): 1625,
  ('20725',): 1218,
  ('21212',): 1315,
  ('22423',): 1653}}

In [ ]:
#Ans 3
rules2

[]

In [ ]:
# Find reference stock code description

stock_df = df_uk[['StockCode', 'Description']].drop_duplicates(ignore_index=True).set_index('StockCode')
stock_df

,Description
StockCode,
85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS
79323P,PINK CHERRY LIGHTS
79323W,WHITE CHERRY LIGHTS
22041,"RECORD FRAME 7"" SINGLE SIZE"
21232,STRAWBERRY CERAMIC TRINKET BOX
...,...
22937,BAKING MOULD CHOCOLATE CUPCAKES
22936,BAKING MOULD ROSE WHITE CHOCOLATE
22935,BAKING MOULD ROSE MILK CHOCOLATE


In [ ]:
# rules is the association rules from the apriori algorithm

for i, rule in enumerate(rules):
    print('Rule {}'.format(i+1))

    lhs = '/'.join(stock_df.loc[list(rule.lhs)].values.reshape(-1)) # antecedent
    rhs = '/'.join(stock_df.loc[list(rule.rhs)].values.reshape(-1)) # consequent

    # antecedent -> consequent
    print('{} -> {}'.format(lhs, rhs))

    print('Supp = {}, Conf = {}, Life = {}, Conv = {}'.format(rule.support, rule.confidence, rule.lift, rule.conviction))
    print()

Rule 1
EDWARDIAN PARASOL NATURAL -> EDWARDIAN PARASOL BLACK
Supp = 0.011079558679763513, Conf = 0.5107296137339056, Life = 23.746716087917804, Conv = 1.9999015817284251

Rule 2
EDWARDIAN PARASOL BLACK -> EDWARDIAN PARASOL NATURAL
Supp = 0.011079558679763513, Conf = 0.5151515151515151, Life = 23.746716087917804, Conv = 2.0177569671153184

Rule 3
EDWARDIAN PARASOL RED -> EDWARDIAN PARASOL BLACK
Supp = 0.010381267166333039, Conf = 0.6947040498442367, Life = 32.30073094091786, Conv = 3.205062555471331

Rule 4
EDWARDIAN PARASOL PINK -> EDWARDIAN PARASOL NATURAL
Supp = 0.00567943764256785, Conf = 0.6931818181818182, Life = 31.95330179477175, Conv = 3.188554215823091

Rule 5
EDWARDIAN PARASOL RED -> EDWARDIAN PARASOL NATURAL
Supp = 0.007960523253107397, Conf = 0.5327102803738317, Life = 24.556114877060686, Conv = 2.0935757135961874

Rule 6
GREEN SPOTTY BOWL/GREEN POLKADOT BOWL -> BLUE SPOTTY BOWL/BLUE POLKADOT BOWL
Supp = 0.0060053070155020715, Conf = 0.7087912087912088, Life = 57.67251498501